In [1]:
import sys
from os import path
import numpy as np
from tqdm import tqdm
from gensim.models import Word2Vec
from model.Embeder import SentenceEmbeder
from sklearn.metrics.pairwise import cosine_similarity


In [2]:
if __package__ is None:
    current_path = path.dirname(path.dirname(path.abspath(
        '/home/fhdufhdu/vscode/KoreanHistoryProject/keyword_extractor/keyword_extractor_ver_W2V.ipynb')))
    sys.path.append(current_path)
    from util.func import save_pickle, load_pickle, get_tokenized_sentences, remove_duplicated_keyword, save_json
else:
    from ..util.func import save_pickle, load_pickle, get_tokenized_sentences, remove_duplicated_keyword, save_json


In [3]:
title_list, sent_list = get_tokenized_sentences('.', True)
doc_list = load_pickle('data/document_list.pickle')
title_list_with_docs = load_pickle('data/title_list.pickle')

In [4]:
embedding_model = Word2Vec(sent_list, size=100, window = 5, min_count=1, workers=4, iter=50, sg=1)
save_pickle('w2v_model/w2v.pickle', embedding_model)

[gensim.models.word2vec] collecting all words and their counts
[gensim.models.word2vec] PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
[gensim.models.word2vec] PROGRESS: at sentence #10000, processed 456931 words, keeping 90764 word types
[gensim.models.word2vec] PROGRESS: at sentence #20000, processed 939021 words, keeping 159697 word types
[gensim.models.word2vec] collected 183728 word types from a corpus of 1104286 raw words and 22162 sentences
[gensim.models.word2vec] Loading a fresh vocabulary
[gensim.models.word2vec] effective_min_count=1 retains 183728 unique words (100% of original 183728, drops 0)
[gensim.models.word2vec] effective_min_count=1 leaves 1104286 word corpus (100% of original 1104286, drops 0)
[gensim.models.word2vec] deleting the raw counts dictionary of 183728 items
[gensim.models.word2vec] sample=0.001 downsamples 19 most-common words
[gensim.models.word2vec] downsampling leaves estimated 1048255 word corpus (94.9% of prior 1104286)
[gensim.mo

In [5]:
w2v_model = load_pickle('w2v_model/w2v.pickle')
bert_embeder = SentenceEmbeder(10)

using cached model
using cached model
using cached model


In [6]:
def get_keyword(title, words, w2v, bert):
    #bert
    _, title_vector = bert.encode([title])
    _, noun_vectors = bert.encode(words)
    title_vector = title_vector[0]

    title_embedding_ = [[value.item() for value in title_vector]]
    candicates_embedding_ = [[value.item() for value in noun_vector]
                            for noun_vector in noun_vectors]
    distances = cosine_similarity(title_embedding_, candicates_embedding_)
    b_similar_list = [(words[i], sim) for i, sim in enumerate(distances[0])]

    #word2vec
    w_similar_list = []
    for word in words:
        w_similar_list.append(
            (word, w2v.wv.similarity(w1=title_list[0], w2=word)))

    final_list = []
    for (w_t, w_s), (_, b_s) in zip(w_similar_list, b_similar_list):
        final_list.append((w_t, w_s + b_s))

    final_list = sorted(final_list, key=lambda sim: sim[1], reverse=True)
    
    return final_list


In [7]:
# qas_dict = {}
# qas_dict['data'] = []d
# doc_idx = 0
# sent_idx = 0
# while sent_idx < len(sent_list) and doc_idx < len(doc_list):
#     title = title_list[sent_idx]
#     title_with_doc = title_list_with_docs[doc_idx]
#     sent = sent_list[sent_idx]
#     doc = doc_list[doc_idx]
#     if title != title_with_doc:
#         doc_idx += 1
#         continue
#     keywords = get_keyword(title, remove_duplicated_keyword(
#         sent), w2v_model, bert_embeder)[:5]
#     for keyword, _ in keywords:
#         qas_set = {}
#         qas_set['doc'] = doc
#         qas_set['answer'] = keyword
#         qas_dict['data'].append(qas_set)
#     sent_idx += 1
#     print('progress :', sent_idx / len(sent_list))

# save_json('data/as_set.json', qas_dict)

